In [1]:
import numpy as np
from manim import *

Manim Community v0.17.3

### Quick test of the sorting function

In [2]:
def ICBICsort(A : list) -> list:
    "The “I Can't Believe It Can Sort” Algorithm"
    n = len(A)

    for i in range(n):
        for j in range(n):
            if A[i] < A[j]:
                A[i], A[j] = A[j], A[i]
                
    return A


In [3]:
N = 10
randomlist = np.random.choice(np.arange(N), size=N, replace=False)

print('Before sorting:')
print(randomlist)
print('After sorting:')
print(ICBICsort(randomlist))

Before sorting:
[8 3 2 1 7 4 6 0 9 5]
After sorting:
[0 1 2 3 4 5 6 7 8 9]


the list is indeed sorted

### Animation

In [4]:
%%manim -ql -v WARNING Sort

def Scene_getvars():
    """Get the main arrays for the animation"""
    # VARIABLES
    N = 6 # number of elements of the list

    # list: (1,2,3,...,N) randomly shuffled
    A = np.random.choice(np.arange(1,N+1), size=N, replace=False)

    heights = 3*A/N
    xs = np.linspace(-6,6,N)  # x-coordinates of each element
    ys = heights/2 - 2.4      # y-coordinates of each element

    return N, A, heights, xs, ys

def Scene_getelems(N, A, heights, xs, ys):
    """Get the elements for the animations"""
    rectangles = [Rectangle(width=6/N, height=h).move_to([x, y, 0]) for h, x, y in zip(heights, xs, ys)]
    values     = [Text(str(value)) for value in A]

    return rectangles, values

def Scene_setup(rectangles, values, xs):
    """Get the animation for setting up the scene"""
    create_recs = tuple([Create(rectangle.set_fill(WHITE, opacity=.6)) for rectangle in rectangles])
    set_values  = tuple([Write(value.move_to([x, -3., 0])) for x, value in zip(xs, values)])

    toplay = create_recs + set_values

    title = Text(str('The \"I Can\'t Believe It Can Sort\" Algorithm')).move_to([0,3,0])

    i, j = 0, 0
    istr = MathTex('i = ' + str(i), font_size=50, color=BLUE).move_to([-5.5,2,0])
    jstr = MathTex('j = ' + str(j), font_size=50, color=YELLOW).next_to(istr, DOWN)

    return AnimationGroup(*toplay), title, istr, jstr

def swap_recs(ys, xs, rectangles, values, l,k):
    """Animation for swapping two elements"""
    y_l, y_k = ys[l], ys[k]
    rectangles[l].generate_target().move_to([xs[k], y_l, 0])
    rectangles[k].generate_target().move_to([xs[l], y_k, 0])
    rec_anim = [MoveToTarget(rectangles[l]), MoveToTarget(rectangles[k])]
    text_anim = [values[l].animate.move_to([xs[k], -3, 0]) , values[k].animate.move_to([xs[l], -3, 0])]
    return AnimationGroup(*tuple(rec_anim + text_anim))

def mark_recs(xs, rectangles, values, l, k, coll = BLUE, colk = YELLOW):
    """Animation for updating the indexes i,j"""
    anim = [rectangle.animate.set_fill(WHITE) for rectangle in rectangles]
    if l == k: # If both loop variable point to the same element, mark it green 
        anim[l] = rectangles[l].animate.set_fill(GREEN)
    else:
        anim[l] = rectangles[l].animate.set_fill(coll)
        anim[k] = rectangles[k].animate.set_fill(colk)

    return AnimationGroup(*tuple(anim))

class Sort(Scene):
    def construct(self):
        # Setup the variables
        N, A, heights, xs, ys = Scene_getvars()

        # Get the animated element
        rectangles, values = Scene_getelems(N, A, heights, xs, ys)

        # SET LAYOUT
        setupanim, title, istr, jstr = Scene_setup(rectangles, values, xs)
        self.add(istr, jstr)
        self.add(title)
        self.play(setupanim)

        # Add an updater for the Texts showing the current indexes
        def update_i(mob):
            mob.become(MathTex('i = ' + str(i), font_size=50, color=BLUE).move_to([-5.5,2,0]))

        def update_j(mob):
            mob.become(MathTex('j = ' + str(j), font_size=50, color=YELLOW).next_to(istr, DOWN))

        istr.add_updater(update_i)
        jstr.add_updater(update_j)

        # I can't believe it can sort algorithm
        for i in range(N):
            for j in range(N):
                self.play(mark_recs(xs, rectangles, values, i,j))
                if A[i] < A[j]:
                    self.play(swap_recs(ys, xs, rectangles, values, i,j))
                    A[i], A[j] = A[j], A[i]
                    rectangles[i], rectangles[j] = rectangles[j], rectangles[i]
                    values[i], values[j] = values[j], values[i]
                    ys[i], ys[j] = ys[j], ys[i]

        # Final animation
        self.play(AnimationGroup(*tuple([rectangle.animate.set_fill(GREEN) for rectangle in rectangles])))
        self.wait()


 